# Read big data with Chunk method

## Data
ows-raw.txt can be download from Baidu Netdisk (2.35G)

Link: https://pan.baidu.com/s/134rHmP5weA54JkCYY4HHvA?pwd=2233   
Extraction code：2233 

In [1]:
import pandas as pd
import time

In [2]:
file_path = 'E:/2024计算社会科学夏校/data/ows-raw.txt'

In [3]:
# Set a dataframe reader object
chunkSize = 10**4
chunk_reader = pd.read_csv(file_path, sep=',', quotechar='"', 
                           iterator=True, on_bad_lines='skip', chunksize=chunkSize)

total_rows = 0
for chunk in chunk_reader:
    total_rows += len(chunk)
    
print(f'Total number of rows: {total_rows}')

Total number of rows: 6602120


In [4]:
chunk.head()

,Twitter ID,Text,Profile Image URL,Day,Hour,Minute,Created At,Geo,From User,From User ID,Language,To User,To User ID,Source
6600000,170726983490211841,Stand Up Mr. US Business Man and take responsi...,http://a2.twimg.com/profile_images/1752607483/...,2012-02-18,4,30,2012-02-18 04:30:59,N;,bentley_cat,463108759,en,NaN,0,&lt;a href=&quot;http://www.bestoftheinternets...
6600001,170727024841854976,RT @C0d3Fr0sty: MT( Link shortened) @Kaymee: I...,http://a2.twimg.com/profile_images/1599465487/...,2012-02-18,4,31,2012-02-18 04:31:09,N;,marylouise996S,15380166,en,NaN,0,&lt;a href=&quot;http://www.tweetdeck.com&quot...
6600002,170727037370253312,China had an #ows before everyone else 1989 Ti...,http://a0.twimg.com/profile_images/1302276340/...,2012-02-18,4,31,2012-02-18 04:31:12,N;,dfwlibrarian,17644162,en,NaN,0,&lt;a href=&quot;http://janetter.net/&quot; re...
6600003,170727054361362433,"Currency, Capital and Evolution: - http://t.co...",http://a3.twimg.com/profile_images/1597982571/...,2012-02-18,4,31,2012-02-18 04:31:16,N;,OmniusManifesto,394061184,it,NaN,0,&lt;a href=&quot;http://www.socialoomph.com&qu...
6600004,170727082391900160,Our problems rise much more from Govts corrupt...,http://a0.twimg.com/profile_images/1592676372/...,2012-02-18,4,31,2012-02-18 04:31:23,N;,IndyPolitico,73935439,en,NaN,0,&lt;a href=&quot;http://www.socialoomph.com&qu...


In [5]:
chunk['Text'].iloc[0]

'Stand Up Mr. US Business Man and take responsibility. #ows #occupy #banks #getmoneyout $BAC $GS $wfc $jpm http://t.co/JPVDmGSl'

## Print out the number of tweets and unique users for each day

In [6]:
chunkSize = 10**4

# Set a dataframe reader object
chunk_reader = pd.read_csv(file_path, sep=',', quotechar='"', 
                           iterator=True, on_bad_lines='skip', chunksize=chunkSize)

# Initialize an empty DataFrame to store the group counts
total = pd.DataFrame()

start_time = time.time()
# Iterate over chunks
for chunk in chunk_reader:
    # Group by the specified column and count the observations in each group
    count = chunk.groupby('Day')['From User ID'].size().reset_index(name='Number of Tweets')
    
    # Group by the specified column and calculate the number of unique IDs in each group
    uniq = chunk.groupby('Day')['From User ID'].nunique().reset_index(name='Unique ID')
    
    # Append the group counts to the total group counts
    total = pd.concat([total, count, uniq], ignore_index=True)
    
    # Fill the database with all missing values to 0.
    total = total.fillna(0)

# After processing all chunks, group again to consolidate counts across chunks
total = total.groupby('Day').sum().reset_index()

# Sort the final group counts
total = total.sort_values(by='Day', ascending=True)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"The code runs {elapsed_time} s")
print(total)

The code runs 43.08106780052185 s
            Day  Number of Tweets  Unique ID
0    2011-10-06           49638.0    24832.0
1    2011-10-07           65238.0    33883.0
2    2011-10-08           65949.0    33204.0
3    2011-10-09           65097.0    32201.0
4    2011-10-10           78619.0    41465.0
..          ...               ...        ...
131  2012-02-14           13580.0     5349.0
132  2012-02-15           13248.0     5598.0
133  2012-02-16           12837.0     5083.0
134  2012-02-17           12468.0     4897.0
135  2012-02-18            4859.0     2273.0

[136 rows x 3 columns]


## Small Practice

In [7]:
df = pd.DataFrame()
df['Day'] = [0,1,2,3,3]
df['From User ID'] = [0,2,2,3,3]

In [8]:
df

,Day,From User ID
0,0,0
1,1,2
2,2,2
3,3,3
4,3,3


In [9]:
count = df.groupby('Day')['From User ID'].size().reset_index(name='Number of Tweets')

In [10]:
uniq = df.groupby('Day')['From User ID'].nunique().reset_index(name='Unique ID')

In [11]:
count

,Day,Number of Tweets
0,0,1
1,1,1
2,2,1
3,3,2


In [12]:
uniq

,Day,Unique ID
0,0,1
1,1,1
2,2,1
3,3,1


In [13]:
merged_df = pd.merge(df, count, on='Day')
merged_df = pd.merge(merged_df, uniq, on='Day')

In [14]:
merged_df

,Day,From User ID,Number of Tweets,Unique ID
0,0,0,1,1
1,1,2,1,1
2,2,2,1,1
3,3,3,2,1
4,3,3,2,1


In [15]:
total = pd.concat([df, count, uniq], ignore_index=True)


In [16]:
total

,Day,From User ID,Number of Tweets,Unique ID
0,0,0.0,NaN,NaN
1,1,2.0,NaN,NaN
2,2,2.0,NaN,NaN
3,3,3.0,NaN,NaN
4,3,3.0,NaN,NaN
5,0,NaN,1.0,NaN
6,1,NaN,1.0,NaN
7,2,NaN,1.0,NaN
8,3,NaN,2.0,NaN
9,0,NaN,NaN,1.0
